In [ ]:
"""

chime data is 4gb

downloaded from 

"""

In [139]:
import glob
import pandas as pd
import numpy as np
import re
import json
import pickle

from sklearn.preprocessing import StandardScaler


In [5]:
eval_dev_chunks = glob.glob('data/chime_home/*.csv')

In [6]:
eval_dev_chunks

['data/chime_home/evaluation_chunks_raw.csv',
 'data/chime_home/development_chunks_refined_crossval_dcase2016.csv',
 'data/chime_home/development_chunks_refined.csv',
 'data/chime_home/evaluation_chunks_refined.csv',
 'data/chime_home/development_chunks_raw.csv']

In [36]:
data_folder = 'data/chime_home/'
data_chunks = f'{data_folder}chunks/'

In [10]:
raw_chunks = glob.glob('data/chime_home/chunks/*.csv')
len(raw_chunks)

In [25]:
files = {}
for filename in eval_dev_chunks:
    files[filename.split('/')[-1]] = pd.read_csv(filename, header=None, names=['index', 'filename'])
    

In [26]:
dev_ref_df = files['development_chunks_refined.csv']

In [31]:
dev_ref_df = dev_ref_df.drop(columns=['index'])

In [58]:
results = []
for filename in dev_ref_df['filename']:
    chunk_df = pd.read_csv(f'{data_chunks}{filename}.csv', header=None)
    chunk_t = chunk_df.set_index(0).T
    results.append(chunk_t)


In [59]:
results[0]

,segmentname,chunknumber,framestart,annotation_a1,session_a1,annotation_a2,session_a2,annotation_a3,session_a3,majorityvote,chunkname
1,CR_lounge_220110_0731.s0,27,5184000,cp,4.0,cop,2.0,cop,4.0,cop,CR_lounge_220110_0731.s0_chunk27


In [60]:
concat_df = pd.concat(results)

In [73]:
human_voices_re = re.compile(r'[cmf]') # child male female

In [66]:
concat_df['human_voice'] = concat_df['majorityvote'].apply(lambda x: 1 if human_voices_re.search(x) else 0)

SyntaxError: invalid syntax (<ipython-input-66-012550b79a61>, line 1)

In [90]:
concat_df['human_voice'].value_counts()

1    1314
0     632
Name: human_voice, dtype: int64

In [148]:
concat_df.to_pickle('development_chunks_refined_concat_df.pkl')

In [97]:
# Feature extraction example
import librosa

# Load the example clip
def extract_features(filename):
    y, sr = librosa.load(f'{filename}')

    # Set the hop length; at 22050 Hz, 512 samples ~= 23ms
    hop_length = 512

    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Beat track on the percussive signal
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                                 sr=sr)

    # Compute MFCC features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

    # And the first-order differences (delta features)
    mfcc_delta = librosa.feature.delta(mfcc)

    # Stack and synchronize between beat events
    # This time, we'll use the mean value (default) instead of median
    beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                        beat_frames)

    # Compute chroma features from the harmonic signal
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr)

    # Aggregate chroma features between beat events
    # We'll use the median value of each feature between beat frames
    beat_chroma = librosa.util.sync(chromagram,
                                    beat_frames,
                                    aggregate=np.median)

    # Finally, stack all beat-synchronous features together
    beat_features = np.vstack([beat_chroma, beat_mfcc_delta])
    
    return {
        'mfcc': mfcc,
        'mfcc_delta': mfcc_delta,
        'beat_mfcc_delta': beat_mfcc_delta,
        'chromagram': chromagram,
        'beat_chroma': beat_chroma,
        'beat_features': beat_features
    }

In [133]:
def extract_features_df(df, column_name = 'chunkname'):
    results = {}
    total = df.shape[0]
    count = 0
    for index, row in df.iterrows():
        print('what is left', (count/total) * 100, '%')
        chunkname = row[column_name]
        chunk_filename = f'{data_chunks}{chunkname}{codec}'
        features = extract_features(chunk_filename)
        results[chunkname] = features
        
        count += 1
        
    return results


In [136]:
all_features = extract_features_df(concat_df)

what is left 0.0 %
what is left 0.051387461459403906 %
what is left 0.10277492291880781 %
what is left 0.1541623843782117 %
what is left 0.20554984583761562 %
what is left 0.2569373072970195 %
what is left 0.3083247687564234 %
what is left 0.3597122302158274 %
what is left 0.41109969167523125 %
what is left 0.4624871531346352 %
what is left 0.513874614594039 %
what is left 0.5652620760534429 %
what is left 0.6166495375128468 %
what is left 0.6680369989722508 %
what is left 0.7194244604316548 %
what is left 0.7708119218910586 %
what is left 0.8221993833504625 %
what is left 0.8735868448098663 %
what is left 0.9249743062692704 %
what is left 0.9763617677286742 %
what is left 1.027749229188078 %
what is left 1.079136690647482 %
what is left 1.1305241521068858 %
what is left 1.18191161356629 %
what is left 1.2332990750256936 %
what is left 1.2846865364850977 %
what is left 1.3360739979445015 %
what is left 1.3874614594039054 %
what is left 1.4388489208633095 %
what is left 1.49023638232271

/Users/greg/anaconda3/envs/metis/lib/python3.7/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


what is left 12.076053442959918 %
what is left 12.127440904419322 %
what is left 12.178828365878726 %
what is left 12.23021582733813 %
what is left 12.281603288797534 %
what is left 12.332990750256938 %
what is left 12.384378211716342 %
what is left 12.435765673175746 %
what is left 12.48715313463515 %
what is left 12.538540596094553 %
what is left 12.589928057553957 %
what is left 12.64131551901336 %
what is left 12.692702980472765 %
what is left 12.744090441932169 %
what is left 12.795477903391571 %
what is left 12.846865364850975 %
what is left 12.89825282631038 %
what is left 12.949640287769784 %
what is left 13.001027749229188 %
what is left 13.052415210688592 %
what is left 13.103802672147996 %
what is left 13.1551901336074 %
what is left 13.206577595066804 %
what is left 13.257965056526208 %
what is left 13.309352517985612 %
what is left 13.360739979445016 %
what is left 13.41212744090442 %
what is left 13.463514902363825 %
what is left 13.514902363823229 %
what is left 13.56628

what is left 24.61459403905447 %
what is left 24.665981500513876 %
what is left 24.71736896197328 %
what is left 24.768756423432684 %
what is left 24.820143884892087 %
what is left 24.871531346351492 %
what is left 24.922918807810895 %
what is left 24.9743062692703 %
what is left 25.025693730729703 %
what is left 25.077081192189105 %
what is left 25.12846865364851 %
what is left 25.179856115107913 %
what is left 25.23124357656732 %
what is left 25.28263103802672 %
what is left 25.334018499486127 %
what is left 25.38540596094553 %
what is left 25.436793422404936 %
what is left 25.488180883864338 %
what is left 25.539568345323744 %
what is left 25.590955806783143 %
what is left 25.642343268242552 %
what is left 25.69373072970195 %
what is left 25.74511819116136 %
what is left 25.79650565262076 %
what is left 25.84789311408017 %
what is left 25.899280575539567 %
what is left 25.95066803699897 %
what is left 26.002055498458375 %
what is left 26.053442959917778 %
what is left 26.10483042137

what is left 37.25590955806783 %
what is left 37.30729701952723 %
what is left 37.35868448098664 %
what is left 37.410071942446045 %
what is left 37.461459403905444 %
what is left 37.51284686536485 %
what is left 37.564234326824256 %
what is left 37.61562178828366 %
what is left 37.66700924974306 %
what is left 37.718396711202466 %
what is left 37.76978417266187 %
what is left 37.82117163412128 %
what is left 37.87255909558068 %
what is left 37.92394655704008 %
what is left 37.97533401849948 %
what is left 38.026721479958894 %
what is left 38.07810894141829 %
what is left 38.1294964028777 %
what is left 38.1808838643371 %
what is left 38.23227132579651 %
what is left 38.28365878725591 %
what is left 38.335046248715315 %
what is left 38.386433710174714 %
what is left 38.43782117163413 %
what is left 38.489208633093526 %
what is left 38.54059609455293 %
what is left 38.59198355601233 %
what is left 38.643371017471736 %
what is left 38.69475847893114 %
what is left 38.74614594039055 %
wha

what is left 49.9486125385406 %
what is left 50.0 %
what is left 50.051387461459406 %
what is left 50.102774922918805 %
what is left 50.15416238437821 %
what is left 50.205549845837616 %
what is left 50.25693730729702 %
what is left 50.30832476875642 %
what is left 50.35971223021583 %
what is left 50.41109969167523 %
what is left 50.46248715313464 %
what is left 50.51387461459404 %
what is left 50.56526207605344 %
what is left 50.61664953751285 %
what is left 50.668036998972255 %
what is left 50.719424460431654 %
what is left 50.77081192189106 %
what is left 50.822199383350465 %
what is left 50.87358684480987 %
what is left 50.92497430626926 %
what is left 50.976361767728676 %
what is left 51.02774922918808 %
what is left 51.07913669064749 %
what is left 51.13052415210688 %
what is left 51.181911613566285 %
what is left 51.2332990750257 %
what is left 51.284686536485104 %
what is left 51.336073997944496 %
what is left 51.3874614594039 %
what is left 51.438848920863315 %
what is left 51

what is left 62.69270298047277 %
what is left 62.744090441932165 %
what is left 62.79547790339157 %
what is left 62.84686536485098 %
what is left 62.89825282631038 %
what is left 62.94964028776978 %
what is left 63.00102774922919 %
what is left 63.052415210688594 %
what is left 63.10380267214799 %
what is left 63.1551901336074 %
what is left 63.206577595066804 %
what is left 63.25796505652621 %
what is left 63.30935251798561 %
what is left 63.360739979445015 %
what is left 63.41212744090442 %
what is left 63.463514902363826 %
what is left 63.514902363823225 %
what is left 63.56628982528263 %
what is left 63.61767728674204 %
what is left 63.66906474820144 %
what is left 63.72045220966084 %
what is left 63.77183967112025 %
what is left 63.82322713257965 %
what is left 63.87461459403906 %
what is left 63.92600205549846 %
what is left 63.977389516957864 %
what is left 64.02877697841727 %
what is left 64.08016443987667 %
what is left 64.13155190133607 %
what is left 64.18293936279548 %
what

what is left 75.53956834532374 %
what is left 75.59095580678314 %
what is left 75.64234326824256 %
what is left 75.69373072970195 %
what is left 75.74511819116135 %
what is left 75.79650565262077 %
what is left 75.84789311408016 %
what is left 75.89928057553958 %
what is left 75.95066803699896 %
what is left 76.00205549845838 %
what is left 76.05344295991779 %
what is left 76.10483042137717 %
what is left 76.15621788283659 %
what is left 76.207605344296 %
what is left 76.2589928057554 %
what is left 76.3103802672148 %
what is left 76.3617677286742 %
what is left 76.41315519013361 %
what is left 76.46454265159302 %
what is left 76.5159301130524 %
what is left 76.56731757451182 %
what is left 76.61870503597122 %
what is left 76.67009249743063 %
what is left 76.72147995889003 %
what is left 76.77286742034943 %
what is left 76.82425488180884 %
what is left 76.87564234326825 %
what is left 76.92702980472764 %
what is left 76.97841726618705 %
what is left 77.02980472764645 %
what is left 77.

what is left 88.38643371017471 %
what is left 88.43782117163413 %
what is left 88.48920863309353 %
what is left 88.54059609455292 %
what is left 88.59198355601234 %
what is left 88.64337101747174 %
what is left 88.69475847893115 %
what is left 88.74614594039055 %
what is left 88.79753340184995 %
what is left 88.84892086330936 %
what is left 88.90030832476874 %
what is left 88.95169578622816 %
what is left 89.00308324768757 %
what is left 89.05447070914697 %
what is left 89.10585817060637 %
what is left 89.15724563206578 %
what is left 89.20863309352518 %
what is left 89.26002055498459 %
what is left 89.31140801644398 %
what is left 89.36279547790339 %
what is left 89.4141829393628 %
what is left 89.4655704008222 %
what is left 89.5169578622816 %
what is left 89.568345323741 %
what is left 89.61973278520041 %
what is left 89.67112024665983 %
what is left 89.72250770811921 %
what is left 89.77389516957862 %
what is left 89.82528263103802 %
what is left 89.87667009249743 %
what is left 89

In [141]:

with open(f'{data_folder}development_chunks_refined_features.pkl', 'wb') as outfile:
    pickle.dump(all_features, outfile)
#     json.dump(all_features, outfile, indent=4)

In [142]:

codec ='.48kHz.wav'
chunkname = 'CR_lounge_220110_0731.s0_chunk27'
chunk_filename = f'{data_chunks}{chunkname}{codec}'
features_one = extract_features(chunk_filename)


In [111]:
from sklearn.decomposition import PCA

variance_explained = 0.6
pca = PCA(variance_explained)
principalComponents = pca.fit_transform(features_one['mfcc_delta'])
# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])

def reduce_dimensions():
    pass

In [ ]:
scaler = StandardScaler()

mfcc_df = pd.DataFrame(np.flatten(features_one['mfcc']))




In [ ]:
mfcc_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=None)